## Two phase flow.
### pezoflow equation.

$$\frac{\partial \rho_w S_w m}{\partial t} + \frac{\partial}{\partial x}(\rho_w W_w)=0$$
$$\frac{\partial \rho_o S_o m}{\partial t} + \frac{\partial}{\partial x}(\rho_o W_o)=0$$
$$W_w=-\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x}$$
$$W_o=-\frac{k_{ro} k}{\mu_o}\frac{\partial p}{\partial x}$$
$$S_o+S_w=1$$
$$m(p)=m_0+\beta_r*(p-p_0)$$
$$\rho_{w,o}(p) = \rho_{w0, o0}(1 + \beta_{w,o}*(p-p_0))$$

$$x\in[0,L]$$
$$t\in[0,T]$$

*Initial and boundary conditions*
$$p(x, t=0)=p_0$$
$$S_w(x, t=0)=S_{w0}$$
$$W_w(x=0, t)=\frac{Q}{S}$$
$$S_w(x=0, t)=1$$
$$p(x=L, t)=p_0$$

1.
$$\rho_w m \frac{\partial S_w}{\partial t} + \rho_w S_w \frac{\partial m}{\partial t} + m S_w \frac{\partial \rho_w}{\partial t} - \frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$
$$\rho_o m \frac{\partial S_o}{\partial t} + \rho_w S_o \frac{\partial m}{\partial t} + m S_o \frac{\partial \rho_o}{\partial t} - \frac{\partial}{\partial x}(\rho_w\frac{k_{ro} k}{\mu_o}\frac{\partial p}{\partial x})=0 $$

2.
$$ m \frac{\partial S_w}{\partial t} + S_w \beta_r \frac{\partial p}{\partial t} + m S_w \frac{\rho_{w0}}{\rho_w} \beta_w\frac{\partial p}{\partial t} - \frac{1}{\rho_w}\frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$
$$m \frac{\partial S_o}{\partial t} +  S_o \beta_r \frac{\partial p}{\partial t} + m S_o \frac{\rho_{o0}}{\rho_o} \beta_o\frac{\partial p}{\partial t} - \frac{1}{\rho_o}\frac{\partial}{\partial x}(\rho_o\frac{k_{ro} k}{\mu_o}\frac{\partial p}{\partial x})=0 $$


3. summarize
$$\frac{\partial p}{\partial t}(S_w \beta_r+(1-S_w) \beta_r + m S_w \frac{\rho_{w0}}{\rho_w} \beta_w+m (1-S_w) \frac{\rho_{o0}}{\rho_o} \beta_o) - \frac{1}{\rho_w}\frac{\partial}{\partial x}(k(\frac{k_{rw} \rho_w}{\mu_w})\frac{\partial p}{\partial x})-\frac{1}{\rho_o}\frac{\partial}{\partial x}(k(\frac{k_{ro} \rho_o}{\mu_o})\frac{\partial p}{\partial x})=0 $$
$$ m \frac{\partial S_w}{\partial t} + S_w (\beta_r + m \frac{\rho_{w0}}{\rho_w} \beta_w) \frac{\partial p}{\partial t} - \frac{1}{\rho_w}\frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$


3. final
$$\frac{\partial p}{\partial t}(\beta_r + m S_w \frac{\rho_{w0}}{\rho_w} \beta_w+m (1-S_w) \frac{\rho_{o0}}{\rho_o} \beta_o) - \frac{1}{\rho_w}\frac{\partial}{\partial x}(k(\frac{k_{rw} \rho_w}{\mu_w})\frac{\partial p}{\partial x})-\frac{1}{\rho_o}\frac{\partial}{\partial x}(k(\frac{k_{ro} \rho_o}{\mu_o})\frac{\partial p}{\partial x})=0 $$
$$ m \frac{\partial S_w}{\partial t} + S_w (\beta_r + m \frac{\rho_{w0}}{\rho_w} \beta_w) \frac{\partial p}{\partial t} - \frac{1}{\rho_w}\frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve, lsqr
from scipy.optimize import newton_krylov, minimize, fsolve
from scipy.interpolate import interp1d
from sol_plot import solution_plot


def expit(x):
    return 1 / (1 + np.exp(-x))

k_water = lambda x: 0.6*(x)**2
k_oil = lambda x: 0.2*(x-1)**2

dk_water = lambda x: 1.2*x
dk_oil = lambda x: 0.4*(x-1)

plt.plot(np.linspace(0,1,100), k_water(np.linspace(0,1,100)))
plt.plot(np.linspace(0,1,100), k_oil(np.linspace(0,1,100)))

In [49]:
class TwoPhase():
    def __init__(self,
                 mu_h,
                 mu_o,
                 glad,
                 G,
                 mu_water,
                 k,
                 Q,
                 m0,
                 beta_r,
                 rho_w0,
                 beta_w,
                 rho_o0,
                 beta_o,
                 t_0,
                 T,
                 nt,
                 tt,
                 pow_n,
                 alfa_k,
                 x_0,
                 L,
                 nx,
                 p_0,
                 s_0,
                 s_left,
                 s_right,
                 mu_type) -> None:
        self.mu_h = mu_h
        self.mu_o = mu_o
        self.glad = glad
        self.G = G
        self.mu_water = mu_water
        self.k = k
        self.Q = Q
        self.m0 = m0
        self.beta_r = beta_r
        self.rho_w0 = rho_w0
        self.beta_w = beta_w
        self.rho_o0 = rho_o0
        self.beta_o = beta_o
        self.t_0 = t_0
        self.T = T
        self.nt = nt
        self.tt = tt
        self.pow_n = pow_n
        self.x_0 = x_0
        self.L = L
        self.nx = nx
        self.p_0 = p_0
        self.s_0 = s_0
        self.s_left = s_left
        self.s_right = s_right
        self.alfa_k = alfa_k
        self.time = np.linspace(t_0, T, int(nt), dtype=np.float64)
        self.dt = self.time[1]-self.time[0]
        self.tt = self.time.flat[np.abs(self.time - tt).argmin()]
        self.x = np.linspace(0, L, nx, dtype=np.float64)
        self.dx = self.x[1]-self.x[0]
        self.mu_oil_arr = []
        self.grad_p = []
        self.mu_stop = np.zeros_like(self.x)
        if mu_type not in ["mu_stop", "mu_run"]:
            raise NameError
        self.mu_type = mu_type

    def m(self, p):
        return self.m0 + self.beta_r*(p-self.p_0)

    def rho_w(self, p):
        return self.rho_w0*(1+self.beta_w*(p-self.p_0))

    def rho_o(self, p):
        return self.rho_o0*(1+self.beta_o*(p-self.p_0))

    def mu_oil(self, grad, mu_stop):
        grad = abs(grad)
        mu = (self.mu_h-self.mu_o) * expit (self.glad * (-grad + self.G)) + self.mu_o
        if self.mu_type=='mu_stop':
            res = np.where(mu<=1.05*self.mu_o, 1, 0)
            res = np.where(mu_stop+res>=1, 1, 0)
            result = np.where(res==1, self.mu_o, mu)
        else:
            result = mu
        return result

    def rate(self, t):
        n = self.pow_n
        a = self.Q*(self.T-self.t_0) / (self.tt**(n+1)/(n+1) + self.tt**n*(self.T-self.tt))
        return np.where(t<self.tt,a*t**(n),a*self.tt**(n))

    def lam_w(self, s, p):
        return self.k*k_water(s)*self.rho_w(p) / self.mu_water

    def lam_o(self, s, p, grad, mu):
        return self.k*k_oil(s)*self.rho_o(p) / self.mu_oil(grad, mu)

    def coef_dp_dt(self, p, s):
        return self.beta_r + self.m(p)*s*self.beta_w*self.rho_w0/self.rho_w(p)+\
                   self.m(p)*(1-s)*self.beta_o*self.rho_o0/self.rho_o(p)

    def solution_init(self):
        p = np.zeros((self.nt, self.nx), dtype=np.float64)
        p[0, :] = self.p_0

        s = np.zeros((self.nt, self.nx), dtype=np.float64)
        s[0,:] = self.s_0
        s[:,0] = self.s_left
        return p, s

    def residual(self, u_new, u_old, s_old, t_step, dt, dx):
        """Выражение невязки для метода Ньютона-Крылова."""
        N = len(u_new)
        res = np.zeros(N, dtype=np.float64)
        grad = np.gradient(u_new)
        k_left_w = self.lam_w(s_old[1:-1], u_new[1:-1])
        k_right_w = self.lam_w(s_old[2:], u_new[2:])
        k_left_o = self.lam_o(s_old[1:-1], u_new[1:-1], grad[1:-1], self.mu_stop[1:-1])
        k_right_o = self.lam_o(s_old[2:], u_new[2:], grad[2:], self.mu_stop[2:])

        res[1:-1] = self.coef_dp_dt(u_new[1:-1], s_old[1:-1])*(u_new[1:-1] - u_old[1:-1]) / dt - \
                    1/self.rho_w(u_new[1:-1])*(k_right_w*(u_new[2:] - u_new[1:-1]) / dx**2 - k_left_w*(u_new[1:-1] - u_new[:-2]) / dx**2) -\
                    1/self.rho_o(u_new[1:-1])*(k_right_o*(u_new[2:] - u_new[1:-1]) / dx**2 - k_left_o*(u_new[1:-1] - u_new[:-2]) / dx**2)

        # Граничные условия
        res[0] = (u_new[1] - u_new[0]) / dx * k_water(s_old[0]) / self.mu_water*self.k + self.rate(t_step)
        res[-1] = u_new[-1] - self.p_0

        return res

    # Метод Ньютона-Крылова для решения нелинейной системы
    def solve_nonlinear(self, u_old, s_old, t_step, dt, dx):
        """Решение системы уравнений с использованием метода Ньютона-Крылова."""
        u_new_guess = np.copy(u_old)  # начальное предположение
        u_new = newton_krylov(lambda u_new: self.residual(u_new, u_old, s_old, t_step, dt, dx), u_new_guess, x_rtol=1e-4)
        return u_new

    # Параметры фракционного потока
    def fw(self, s, p):
        """ Функция фракционного потока воды """
        grad = np.gradient(p, self.dx)

        return self.lam_w(s, p) * grad

    def exact(self, t):
        Swi0 = self.s_0

        def f(sat):
            return k_water(sat)/(k_water(sat)+self.mu_water/self.mu_o*k_oil(sat))

        def df(sat):
            return (dk_water(sat)*(k_water(sat)+self.mu_water/self.mu_o*k_oil(sat))-k_water(sat)*(dk_water(sat)+self.mu_water/self.mu_o*dk_oil(sat)))/(k_water(sat)+self.mu_water/self.mu_o*k_oil(sat))**2

        def opt(sat):
            return f(sat)-f(Swi0)-df(sat)*(sat-Swi0)

        Sc = fsolve(opt, 0.5)
        Xc = self.Q/self.m0*(f(Sc)-f(Swi0))/(Sc-Swi0)*t
        Sx1 = np.zeros(200)
        X1 = np.zeros(200)
        Sx2 = np.zeros(200)
        X2 = np.zeros(200)
        Sx1[0] = 1
        Sx2[0] = Swi0
        X2[0] = Xc
        for i in range(1,200):
            Sx1[i] = Sx1[i-1]-(1-Sc)/199 
            Sx2[i] = Swi0
            X2[i] = X2[i-1] + (self.L+1e-3-Xc)/199
        for i in range(1,200):
            X1[i] = self.Q/self.m0 * df(Sx1[i]) * t
        X3 = np.array([Xc[0], Xc[0]])
        Sx3 = np.array([Sc, Swi0])
        return np.concatenate((X1,X3,X2)), np.concatenate((Sx1,Sx3,Sx2))

    def solve(self):
        p, s = self.solution_init()
        s_buckley = np.zeros_like(s)
        s_buckley[0,:] = s[0,:]
        mu_oil_init = np.zeros(self.nx, dtype=np.float64)+self.mu_h
        self.mu_oil_arr.append(mu_oil_init)
        self.grad_p.append(np.gradient(p[0,:], self.dx))
        # Решение системы с использованием Ньютона-Крылова
        for t, t_step in enumerate(self.time[:-1]):
            # решение уравнения для давления
            p_old = p[t, :].copy()
            s_old = s[t, :].copy()
            p_new = self.solve_nonlinear(p_old, s_old, t_step, self.dt, self.dx)
            p[t+1, :] = p_new.copy()
            grad_new = np.gradient(p_new, self.dx)
            # решение уравнения для насыщенности
            fw_val = self.fw(s_old, p_new)
            s_new = np.zeros_like(s[t,:], dtype=np.float64)
            s_new[1:-1] = s_old[1:-1] + self.dt/(self.rho_w(p_new[1:-1])*self.m(p_new[1:-1])*self.dx) * (fw_val[1:-1] - fw_val[:-2]) - s_old[1:-1]*(self.beta_r/self.m(p_new[1:-1]) + self.beta_w*self.rho_w0/self.rho_w(p_new[1:-1]))*(p_new[1:-1]-p_old[1:-1])
            s_new[0] = self.s_left
            s_new[-1] = s_new[-2]
            s[t+1,:] = s_new.copy()
            
            if t>=0:
                mu_oil_new = self.mu_oil(grad_new, self.mu_stop)
                res = np.where(mu_oil_new==self.mu_o, 1, 0)
                self.mu_stop = np.where(self.mu_stop+res>=1, 1, 0)
            self.mu_oil_arr.append(mu_oil_new)
            self.grad_p.append(grad_new)
            x_b, s_b = self.exact(t_step)
            func = interp1d(x_b, s_b, kind='nearest')
            s_b = func(self.x)
            s_buckley[t+1, :] = s_b
        self.mu_oil_arr = np.array(self.mu_oil_arr)
        self.grad_p = np.array(self.grad_p)
        return p, s, x_b, s_buckley

In [ ]:
solver = TwoPhase(mu_h = 2e-3,
                mu_o = 2e-3,
                glad = 0.00002,
                G = 0.4e7,
                mu_water = 8e-4,
                k = 10e-15,
                Q = 0.4/86400,
                m0 = 0.2,
                beta_r = 0,
                rho_w0 = 990.,
                beta_w = 5e-10,
                rho_o0 = 800.,
                beta_o = 8e-10,
                t_0 = 0,
                T = 10*86400,
                nt = 150*5,
                tt = 0.001*864000,
                pow_n = 2,
                alfa_k=1e-9,
                x_0 = 0,
                L = 100,
                nx = 500,
                p_0 = 1e+6,
                s_0 = 0.,
                s_left = 1,
                s_right = 1,
                mu_type = "mu_stop")

p, s, x_b, s_b = solver.solve()

In [ ]:
solution_plot([[solver.x, p],
               [solver.x, s],
               [solver.x, s_b],
               [solver.x, solver.mu_oil_arr],
               [solver.x, solver.grad_p]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)"),
               idx=[[1,1], [1,2], [1,2], [2,1], [2,2], [2,2]],
               nt=solver.nt,
               height=800,
               width=1200)


In [4]:
mu_h = 5e-3
mu_o = 2e-3
glad = 0.00005
G = 4.4e6

mu_water = 8e-4

k = 10e-15
Q = 0.4/86400

t_0 = 0
T = 1*86400
nt = 1500
time = np.linspace(t_0, T, nt)
dt = time[1]-time[0]
tt = 0.3*86400
tt = time.flat[np.abs(time - tt).argmin()]
pow_n = 4

x_0 = 0
L = 1
nx = 100
x = np.linspace(0, L, nx)

dx = x[1]-x[0]

p0 = 1e+6
m0 = 0.2

beta_r = 1e-10

rho_w0 = 1000
beta_w = 0

rho_o0 = 800
beta_o = 0

In [ ]:


plt.plot(np.linspace(4e+6, 5e+6, nx), mu_oil(np.linspace(4e+6, 5e+6, nx), mu_oil_x))
plt.show()

def rate(t):
    n = pow_n
    a = Q*(T-t_0) / (tt**(n+1)/(n+1) + tt**n*(T-tt))
    return np.where(t<tt,a*t**(n),a*tt**(n))

plt.plot(time, rate(time))
plt.show()

def m(p):
    return m0 + beta_r*(p-p0)

def rho_w(p):
    return rho_w0*(1+beta_w*(p-p0))

def rho_o(p):
    return rho_o0*(1+beta_o*(p-p0))

k_water = lambda x: 0.6*(x)**2
k_oil = lambda x: 0.2*(x-1)**2

plt.plot(np.linspace(0, 1, 30), k_water(np.linspace(0, 1, 30)))
plt.plot(np.linspace(0, 1, 30), k_oil(np.linspace(0, 1, 30)))
plt.show()

$$m\frac{\partial S_w}{\partial t}(\rho_w -\rho_o) + \frac{\partial p}{\partial t}(\rho_w S_w \beta_r+\rho_o S_o \beta_r + m S_w \rho_{w0} \beta_w+m S_o \rho_{o0} \beta_o) - \frac{\partial}{\partial x}(k(\frac{k_{rw} \rho_w}{\mu_w}+\frac{k_{ro} \rho_o}{\mu_o})\frac{\partial p}{\partial x})=0 $$
$$\rho_w m \frac{\partial S_w}{\partial t} + \frac{\partial p}{\partial t}(\rho_w S_w \beta_r + m S_w \rho_{w0} \beta_w) - \frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$

$$\frac{\partial S_w}{\partial t} + \frac{\partial p}{\partial t}(S_w \beta_r /m + S_w \rho_{w0}/\rho_w \beta_w) - \frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})/\rho_w m=0 $$

In [ ]:
# Сетка по x
x = np.linspace(0, L, nx)
mu_oil_x = np.zeros(nx)+mu_h
# Инициализация давления
p = np.zeros((nt, nx))

p[0, :] = p0

# Инициализация насыщенности
s = np.zeros((nt, nx))

s[0,:] = 0.
s[:,0] = 1

# Функция проводимости
def lam_w(s,p):
    return rho_w(p)*k_water(s) / mu_water

def lam_o(s, p, grad, mu_oil_x):
    return rho_o(p)*k_oil(s) / mu_oil(grad, mu_oil_x)

def beta(s, p, grad, mu_oil_x):
    return (lam_o(s, p, grad, mu_oil_x) + lam_w(s, p)) * k
    

def residual(u_new, u_old, s_old, s_old1, mu, t, dt, dx):
    """Выражение невязки для метода Ньютона-Крылова."""
    N = len(u_new)
    res = np.zeros(N)
    grad = np.gradient(u_new, x)
    k_left = (beta(s_old1[:-2], u_new[:-2], grad[:-2], mu[:-2]) + beta(s_old1[1:-1], u_new[1:-1], grad[1:-1], mu[1:-1])) / 2
    k_right = (beta(s_old1[1:-1], u_new[1:-1], grad[1:-1], mu[1:-1]) + beta(s_old1[2:], u_new[2:], grad[2:], mu[2:])) / 2
    res[1:-1] = m(u_new[1:-1])*(rho_w(u_new[1:-1])-rho_o(u_new[1:-1]))*(s_old1[1:-1] - s_old[1:-1]) / dt +\
        (rho_w(u_new[1:-1])*s_old1[1:-1]*beta_r + rho_o(u_new[1:-1])*(1-s_old1[1:-1])*beta_r +\
         m(u_new[1:-1])*s_old1[1:-1]*rho_w0*beta_w + m(u_new[1:-1])*(1-s_old1[1:-1])*rho_o0*beta_o)*(u_new[1:-1] - u_old[1:-1]) / dt -\
         (k_right * (u_new[2:] - u_new[1:-1]) / dx**2 - k_left * (u_new[1:-1] - u_new[:-2]) / dx**2)

    # Граничные условия (Дирихле)
    res[0] = (u_new[1] - u_new[0]) / dx * k_water(s_old1[0]) / mu_water*k + rate(t)
    res[-1] = u_new[-1] - p0
    return res

# Метод Ньютона-Крылова для решения нелинейной системы
def solve_nonlinear(u_old, s_old, s_old1, mu, t, dt, dx):
    """Решение системы уравнений с использованием метода Ньютона-Крылова."""
    u_new_guess = np.copy(u_old)  # начальное предположение
    u_new = newton_krylov(lambda u_new: residual(u_new, u_old, s_old, s_old1, mu, t, dt, dx), u_new_guess, f_tol=2e-6)
    return u_new


# Параметры фракционного потока
def fw(s, p, x):
    """ Функция фракционного потока воды """
    grad = np.gradient(p, x)

    return lam_w(s, p) * k * grad

# Решение системы с использованием Ньютона-Крылова
for t, t_step in enumerate(time[:-1]):
    # решение уравнения для давления
    p_old = p[t, :].copy()
    s_old1 = s[t, :].copy()
    if t!=0:
        s_old = s[t-1, :].copy()
    else:
        s_old = s[t, :].copy()

    p_new = solve_nonlinear(p_old, s_old, s_old1, mu_oil_x, t_step, dt, dx)
    p[t+1, :] = p_new

    # решение уравнения для насыщенности
    fw_val = fw(s_old1, p_new, x)

    s_new = np.zeros_like(s[t,:])
    s_new[1:] = s_old1[1:] - (p_new[1:] - p_old[1:])*(s_old1[1:]*beta_r/m(p_new[1:])+\
                                                     s_old1[1:]*rho_w0/rho_w(p_new[1:])*beta_w) +\
                dt/dx * (fw_val[1:] - fw_val[:-1]) / (rho_w(p_new[1:])*m(p_new[1:]))

    s_new[0] = 1
    s[t+1,:] = s_new
    if t!=0:
        mu_oil_x = mu_oil(np.gradient(p_new, x), mu_oil_x)
    if t%100==0:
        # plt.plot(x, p_new, label='p')
        # plt.legend()
        # plt.show()
        plt.plot(x, mu_oil_x, label='mu_oil')
        plt.legend()
        plt.show()
        plt.plot(x, np.gradient(p_new, x), x, -np.ones_like(x)*G, label='grad p')
        plt.legend()
        plt.show()
        plt.plot(x, s_new, '--')
        plt.show()
